**Effi_CutMix.py**

**.. ├ cutmix**

**…. ├ cutmix.py**

**…. └ utils.py**

.. ├ models

.. ├ train_imgs

.. ├ test_imgs

.. ├ train.csv

.. ├ test.csv

.. └ sample_submission.csv

# 0. cutmix 폴더 내 코드

> cutmix.py



In [ ]:
import numpy as np
import random
from torch.utils.data.dataset import Dataset

from cutmix.utils import onehot, rand_bbox

class CutMix(Dataset):
    def __init__(self, dataset, num_class, num_mix=1, beta=1., prob=1.0):
        self.dataset = dataset
        self.num_class = num_class
        self.num_mix = num_mix
        self.beta = beta
        self.prob = prob

    def __getitem__(self, index):
        img = self.dataset[index]['img']
        lb = self.dataset[index]['label']
        lb_onehot = onehot(self.num_class, lb)

        for _ in range(self.num_mix):
            r = np.random.rand(1)
            if self.beta <= 0 or r > self.prob:
                continue

            # generate mixed sample
            lam = np.random.beta(self.beta, self.beta)
            rand_index = random.choice(range(len(self)))

            img2 = self.dataset[rand_index]['img']
            lb2 = self.dataset[rand_index]['label']
            lb2_onehot = onehot(self.num_class, lb2)

            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            img[:, bbx1:bbx2, bby1:bby2] = img2[:, bbx1:bbx2, bby1:bby2]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
            lb_onehot = lb_onehot * lam + lb2_onehot * (1. - lam)

        return {'img':img,'label': lb_onehot}

    def __len__(self):
        return len(self.dataset)



> utils.py



In [ ]:
import numpy as np
import torch
from torch.nn.modules.module import Module


class CutMixCrossEntropyLoss(Module):
    def __init__(self, size_average=True):
        super().__init__()
        self.size_average = size_average

    def forward(self, input, target,class_weight):
        if len(target.size()) == 1:
            target = torch.nn.functional.one_hot(target, num_classes=input.size(-1))
            target = target.float().cuda()
        return cross_entropy(input, target, self.size_average,class_weight)


def cross_entropy(input, target, size_average=True,class_weight=None):
    """ Cross entropy that accepts soft targets
    Args:
         pred: predictions for neural network
         targets: targets, can be soft
         size_average: if false, sum is returned instead of mean
         class_weight: weights, for weighted Cross Entropy
    Examples::
        input = torch.FloatTensor([[1.1, 2.8, 1.3], [1.1, 2.1, 4.8]])
        input = torch.autograd.Variable(out, requires_grad=True)
        target = torch.FloatTensor([[0.05, 0.9, 0.05], [0.05, 0.05, 0.9]])
        target = torch.autograd.Variable(y1)
        loss = cross_entropy(input, target)
        loss.backward()
    """
    if class_weight==None:
        class_weight = torch.ones([target.shape[1]], dtype=torch.float32, device='cuda')

    logsoftmax = torch.nn.LogSoftmax(dim=1)
    if size_average:
        return torch.mean(torch.sum(-(target*class_weight) * logsoftmax(input), dim=1)/torch.sum(target*class_weight,dim=1))
    else:
        return torch.sum(torch.sum(-(target*class_weight) * logsoftmax(input), dim=1)/torch.sum(target*class_weight,dim=1))


def onehot(size, target):
    vec = torch.zeros(size, dtype=torch.float32)
    vec[target] = 1.
    return vec


def rand_bbox(size, lam):
    if len(size) == 4:
        W = size[2]
        H = size[3]
    elif len(size) == 3:
        W = size[1]
        H = size[2]
    else:
        raise Exception

    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2


아래 코드를 필요에 맞게 수정하여 사용하였습니다.

Reference: https://github.com/ildoonet/cutmix

# 1. 사용 Library

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import StratifiedKFold
from cutmix.cutmix import CutMix
from cutmix.utils_ import CutMixCrossEntropyLoss

import warnings

warnings.filterwarnings("ignore")

# 2. DataSet and Data augmentation

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, files, labels=None, mode='train', transform=None):
        self.mode = mode
        self.files = files
        if mode == 'train':
            self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        if self.mode == 'train':
            img = Image.open('train_imgs/' + self.files[i])

            if self.transform:
                img = self.transform(img)

            return {
                'img': torch.tensor(img, dtype=torch.float32).clone().detach(),
                'label': torch.tensor(self.labels[i], dtype=torch.long)
            }
        else:
            img = Image.open('test_imgs/' + self.files[i])
            if self.transform:
                img = self.transform(img)

            return {
                'img': torch.tensor(img, dtype=torch.float32).clone().detach(),
            }

학습 시에만 데이터를 증강하도록 하기 위해 Transform을 두가지로 하였습니다.


In [ ]:
mytransform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(244),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(0, 360)),
    transforms.RandomPerspective(),
    transforms.ToTensor(),
])

myvaltransform =transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# 3. Model

In [ ]:
class CNN_Model(nn.Module):
    def __init__(self, class_n, rate=0.2):
        super(CNN_Model, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b7')
        self.dropout = nn.Dropout(rate)
        self.output_layer = nn.Linear(in_features=1000, out_features=class_n, bias=True)

    def forward(self, inputs):
        output = self.output_layer(self.dropout(self.model(inputs)))
        return output


# 4. Train & Inference

In [ ]:
def train_step(model, batch_item, epoch, batch, training,class_weight=None):
    img = batch_item['img'].to(device)
    label = batch_item['label'].to(device)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label,class_weight=class_weight)
        loss.backward()
        optimizer.step()

        return loss
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label,class_weight=class_weight)
            
def predict(models,dataset):
    for fold,model in enumerate(models):
        model.eval()
    tqdm_dataset = tqdm(enumerate(dataset))
    training = False
    results = []
    for batch, batch_item in tqdm_dataset:
        img = batch_item['img'].to(device)
        for fold,model in enumerate(models):
            with torch.no_grad():
                if fold ==0:
                    output = model(img)
                else:
                    output = output+model(img)
        output = 0.2*output
        output = torch.tensor(torch.argmax(output, axis=-1), dtype=torch.int32).cpu().numpy()
        results.extend(output)
    return results

# 5. Main Sequence

 - Setting

In [ ]:
if __name__ == '__main__':
    # Load Data
    train_total = pd.read_csv('train.csv')
    test = pd.read_csv('test.csv')
    # Hyperparameter Setting
    device = torch.device("cuda")
    batch_size = 8
    class_n = len(train_total['disease_code'].unique())
    learning_rate = 2e-4
    epochs = 300
    folds = 5
    random_seed=100
    save_path = 'models/model.'
    # Cross Validation
    kfold = StratifiedKFold(n_splits=folds,shuffle=True,random_state=random_seed)
    # Loss Weights
    class_weight=torch.FloatTensor(1/train_total['disease_code'].value_counts()).cuda()

    torch.manual_seed(random_seed)

 - Dataset

In [ ]:
    train_dataset = CustomDataset(train_total['img_path'].str.split('/').str[-1].values, train_total['disease_code'].values,
                                  transform=mytransform)
    train_dataset = CutMix(train_dataset,num_class=7,beta=1.0,prob=0.5,num_mix=2)
    valid_dataset = CustomDataset(train_total['img_path'].str.split('/').str[-1].values, train_total['disease_code'].values,
                                  transform=myvaltransform)
    test_dataset = CustomDataset(test['img_path'].str.split('/').str[-1], labels=None, mode='test',
                                 transform=myvaltransform)

    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=3, shuffle=False)

 - Train


In [ ]:
    k_loss_plot, k_val_loss_plot = [], []    
    for fold,(train_idx,valid_idx) in enumerate(kfold.split(train_dataset,train_total["disease_code"])):
        train_subsampler = SubsetRandomSampler(train_idx)
        valid_subsampler = SubsetRandomSampler(valid_idx)

        train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=3, sampler=train_subsampler)
        val_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=3, sampler=valid_subsampler)

        model = CNN_Model(class_n).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        criterion = CutMixCrossEntropyLoss(True)

        loss_plot, val_loss_plot = [], []
        # Training
        for epoch in range(epochs):
            total_loss, total_val_loss = 0, 0
            tqdm_dataset = tqdm(enumerate(train_dataloader))
            training = True
            for batch, batch_item in tqdm_dataset:
                batch_loss = train_step(model,batch_item, epoch, batch, training,class_weight=class_weight)
                total_loss += batch_loss

                tqdm_dataset.set_postfix({
                    'Epoch': epoch + 1,
                    'Loss': '{:06f}'.format(batch_loss.item()),
                    'Total Loss': '{:06f}'.format(total_loss / (batch + 1))
                })
            loss_plot.append((total_loss / (batch + 1)).cpu().item())

            tqdm_dataset = tqdm(enumerate(val_dataloader))
            training = False
            for batch, batch_item in tqdm_dataset:
                batch_loss = train_step(model,batch_item, epoch, batch, training)
                total_val_loss += batch_loss

                tqdm_dataset.set_postfix({
                    'Epoch': epoch + 1,
                    'Val Loss': '{:06f}'.format(batch_loss.item()),
                    'Total Val Loss': '{:06f}'.format(total_val_loss / (batch + 1))
                })
            val_loss_plot.append((total_val_loss / (batch + 1)).cpu().item())

            if np.min(val_loss_plot) == val_loss_plot[-1]:
                torch.save(model.state_dict(), save_path+str(fold)+".pt")

        k_loss_plot.append(min(loss_plot))
        k_val_loss_plot.append(min(val_loss_plot))

    print("Train Loss: ",np.mean(k_loss_plot),", Valid Loss: ",np.mean(k_val_loss_plot))

 - Inference

In [ ]:
    models = []
    for i in range(5):
        model = CNN_Model(class_n).to(device)
        model.load_state_dict(torch.load(save_path+str(i)+".pt"))
        models.append(model)
    preds = predict(models,test_dataloader)

 - Submission
 

In [ ]:
    submission = pd.read_csv('sample_submission.csv')
    submission.iloc[:, 1] = preds
    submission.to_csv('submission.csv', index=False)